In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
# rcParams['figure.figsize'] =15,9
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

/Users/neerajpadarthi/anaconda3/envs/work/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/neerajpadarthi/anaconda3/envs/work/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/neerajpadarthi/anaconda3/envs/work/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/neerajpadarthi/anaconda3/envs/work/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [10]:
df = pd.read_csv('parking_formatted.csv')

In [11]:
df.head()

,start_time,no_of_cars,summary,temperature
0,2018-09-21 00:00:00,3,Clear,87.95
1,2018-09-21 00:05:00,4,Clear,87.95
2,2018-09-21 00:10:00,4,Clear,87.95
3,2018-09-21 00:15:00,4,Clear,87.95
4,2018-09-21 00:20:00,4,Clear,87.95


In [12]:
df.dtypes

start_time      object
no_of_cars       int64
summary         object
temperature    float64
dtype: object

In [16]:
df['start_time'] = pd.to_datetime(df.start_time,format='%Y/%m/%d').astype('datetime64[s]')

In [17]:
df.head()

,start_time,no_of_cars,summary,temperature
0,2018-09-21 00:00:00,3,Clear,87.95
1,2018-09-21 00:05:00,4,Clear,87.95
2,2018-09-21 00:10:00,4,Clear,87.95
3,2018-09-21 00:15:00,4,Clear,87.95
4,2018-09-21 00:20:00,4,Clear,87.95


In [28]:
df.dtypes

start_time     datetime64[ns]
no_of_cars              int64
summary                object
temperature           float64
dtype: object

In [24]:
data =pd.get_dummies(data = df,columns = [
'summary'],dtype=np.int32 )

In [26]:
data.head()

,start_time,no_of_cars,temperature,summary_Clear,summary_Foggy,summary_Heavy Rain,summary_Light Rain,summary_Light Snow,summary_Mostly Cloudy,summary_Overcast,summary_Partly Cloudy,summary_Rain
0,2018-09-21 00:00:00,3,87.95,1,0,0,0,0,0,0,0,0
1,2018-09-21 00:05:00,4,87.95,1,0,0,0,0,0,0,0,0
2,2018-09-21 00:10:00,4,87.95,1,0,0,0,0,0,0,0,0
3,2018-09-21 00:15:00,4,87.95,1,0,0,0,0,0,0,0,0
4,2018-09-21 00:20:00,4,87.95,1,0,0,0,0,0,0,0,0


In [33]:
data.dtypes
data.index = df.start_time

In [38]:
data.drop(columns={'start_time'},inplace=True)

In [40]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

In [41]:
johan_test_temp = df
coint_johansen(johan_test_temp,-1,1).eig

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [42]:
#creating the train and validation set
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

/Users/neerajpadarthi/anaconda3/envs/work/lib/python3.5/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency 5T will be used.
  % freq, ValueWarning)


In [47]:
#converting predictions to dataframe
from math import sqrt

cols = data.columns
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,11):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#check rmse
for i in cols:
    print('rmse value for', i, 'is : ', sqrt(mean_squared_error(pred[i], valid[i])))

rmse value for no_of_cars is :  2.4566684453383347
rmse value for temperature is :  9.536197302789363
rmse value for summary_Clear is :  0.46671276038794896
rmse value for summary_Foggy is :  0.0173494363833224
rmse value for summary_Heavy Rain is :  0.0011058582160100364
rmse value for summary_Light Rain is :  0.01937387084102008
rmse value for summary_Light Snow is :  0.0016477383561473889
rmse value for summary_Mostly Cloudy is :  0.17665162915943347
rmse value for summary_Overcast is :  0.36105302562549924
rmse value for summary_Partly Cloudy is :  0.18946996710276237
rmse value for summary_Rain is :  0.01582461674082793


In [49]:
#make final predictions
model = VAR(endog=data)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=3)
print(yhat)


[[ 1.87661496e-01  5.06408218e+01  9.94210219e-01  7.92252027e-05
   6.73581508e-05  7.45921384e-05 -4.82929532e-06  9.72163935e-04
   8.32210446e-04  3.41011905e-03  3.58941005e-04]
 [ 3.64473520e-01  5.06385017e+01  9.88551649e-01  1.51199834e-04
   1.28339402e-04  1.56993087e-04 -8.85886766e-06  1.99627465e-03
   1.70402782e-03  6.62631173e-03  6.94063532e-04]
 [ 5.31063275e-01  5.06363573e+01  9.83019239e-01  2.16484260e-04
   1.83444105e-04  2.46319851e-04 -1.21046106e-05  3.06478726e-03
   2.61429490e-03  9.66049885e-03  1.00703656e-03]]


/Users/neerajpadarthi/anaconda3/envs/work/lib/python3.5/site-packages/statsmodels/tsa/base/tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency 5T will be used.
  % freq, ValueWarning)


In [50]:
data.head()

,no_of_cars,temperature,summary_Clear,summary_Foggy,summary_Heavy Rain,summary_Light Rain,summary_Light Snow,summary_Mostly Cloudy,summary_Overcast,summary_Partly Cloudy,summary_Rain
start_time,,,,,,,,,,,
2018-09-21 00:00:00,3,87.95,1,0,0,0,0,0,0,0,0
2018-09-21 00:05:00,4,87.95,1,0,0,0,0,0,0,0,0
2018-09-21 00:10:00,4,87.95,1,0,0,0,0,0,0,0,0
2018-09-21 00:15:00,4,87.95,1,0,0,0,0,0,0,0,0
2018-09-21 00:20:00,4,87.95,1,0,0,0,0,0,0,0,0


In [51]:
data.to_csv('timeseries_summary.csv')